# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json


# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
print(transcript)

                  event                            person  time  \
0        offer received  78afa995795e4d85b5d9ceeca43f5fef     0   
1        offer received  a03223e636434f42ac4c3df47e8bac43     0   
2        offer received  e2127556f4f64592b11af22de27a7932     0   
3        offer received  8ec6ce2a7e7949b1bf142def7d0e0586     0   
4        offer received  68617ca6246f4fbc85e91a2a49552598     0   
5        offer received  389bc3fa690240e798340f5a15918d5c     0   
6        offer received  c4863c7985cf408faee930f111475da3     0   
7        offer received  2eeac8d8feae4a8cad5a6af0499a211d     0   
8        offer received  aa4862eba776480b8bb9c68455b8c2e1     0   
9        offer received  31dda685af34476cad5bc968bdb01c53     0   
10       offer received  744d603ef08c4f33af5a61c8c7628d1c     0   
11       offer received  3d02345581554e81b7b289ab5e288078     0   
12       offer received  4b0da7e80e5945209a1fdddfe813dbe0     0   
13       offer received  c27e0d6ab72c455a8bb66d980963de60     

# Cleaning the Data Part I: Cleaning and combining portfolio and profile data frames

In [3]:
# a row of data for the ML model is going to be everything we know about an 'offer recieved': offer type, profile of offeree
# $ spent by person within timespan of offer

#first, break up offers from transactions
offers = transcript[transcript['event'] != 'transaction'].reset_index()
transactions = transcript[transcript['event'] == 'transaction'].reset_index()

offer_val_index = offers.columns.get_loc('value')

def get_offer(val_dict):
    if 'offer id' in val_dict.keys():
        return val_dict['offer id']
    if 'offer_id' in val_dict.keys():
        return val_dict['offer_id']

#pull data out of the dictionary object
offers['offer_id'] = pd.Series([get_offer(offers.iloc[i,offer_val_index]) for i in range(0,len(offers))])
transactions['amount'] = pd.Series([float(transactions['value'].iloc[i].get('amount')) 
                                    for i in range(0,len(transactions['value']))])

#clean up profile date, need to make the member since attribute and integer (days from min date)
profile['member_date'] = pd.Series([pd.to_datetime(str(i)) for i in profile['became_member_on']])
min_date = min(profile['member_date'])
profile['member_date_int'] = pd.Series([i - min_date for i in profile['member_date']]).dt.days

#make dummy variables for gender
profile = pd.concat([profile, pd.get_dummies(profile['gender'])], axis=1)

#impute values for income, add impute_income flag
mean_income = round(np.mean(profile['income'][profile['income'].notnull()]))

income, income_flag = [],[]
for index, i in enumerate(profile['income'].isnull()):
    if i:
        income.append(mean_income)
        income_flag.append(1)
    else:
        income.append(profile['income'][index])
        income_flag.append(0)
        
profile['income'] = pd.Series(income)
profile['income_impute_flag'] = pd.Series(income_flag)

#remove redundant columns from profile dataframe
profile_merge = profile.drop(['became_member_on','member_date','gender'],axis=1)

#create new columns and extract binary variables for type of channel used
portfolio['web'] = pd.Series([0 for i in range(0,len(portfolio))])
portfolio['email'] = pd.Series([0 for i in range(0,len(portfolio))])
portfolio['mobile'] = pd.Series([0 for i in range(0,len(portfolio))])
portfolio['social'] = pd.Series([0 for i in range(0,len(portfolio))])

for index, i in enumerate(portfolio['channels']):
    for j in i:
        portfolio[j][index] = 1
        
portfolio = pd.concat([portfolio, pd.get_dummies(portfolio['offer_type'])], axis=1)

#merge cleaned up dataframes
offers = offers.merge(portfolio,how='left',left_on='offer_id', right_on='id')
offers = offers.merge(profile_merge,how='left',left_on='person', right_on='id')
offers = offers.reset_index()


offers_recieved = offers[offers['event'] == 'offer received']
offers_recieved = offers_recieved.drop(['level_0'],axis=1).reset_index()


C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Cleaning the Data Part II: Quantifying the Outcomes of Offers

In [4]:
#for each offer recieved derive the start and end time of offer influence                                                                                                                                                                                                                                                                                                  
offers_recieved['influence_start'] = offers_recieved['time']
offers_recieved['influence_end'] = offers_recieved['time'] + offers_recieved['duration']

def spending_in_window(offer_start,offer_end,person,transaction_df):
    """sums a customers spending during a window of an offer
    Inputs:
    offer start: when offer was sent
    offer end: offer start + duration of offer
    person: person offer was sent to
    transaction_df: data frame of all transaction labeled offer completed.   
    returns sum of spending within time window for a given customer.  
    """
    sum_df = transaction_df[(transaction_df['time'] >= offer_start) & (transaction_df['time'] <= offer_end) 
                   & (transaction_df['person'] == person)]['amount'].values
    if len(sum_df) == 0:
        return 0
    else:
        return np.sum(sum_df)
    
def check_viewed(offer_start,offer_end,person,offer_viewed_df):
    """searches transactions to see if they had an offer viewed event in the specified timespan
    for an offer's influence.  
    Inputs:
    offer start: when offer was sent
    offer end: offer start + duration of offer
    person: person offer was sent to
    offer_viewed_df: data frame of all transaction labeled offer_viewed.   
    returns 1 if the offer has been viewed, 0 otherwise.  
    """
    person_offer_df = offer_viewed_df[offer_viewed_df['person'] == person]
    column_index = person_offer_df.columns.get_loc('time')
    output = 0
    if person_offer_df is None:
        return output
    else:
        for i in range(0,len(person_offer_df)):
            if person_offer_df.iloc[i,column_index] >= offer_start and person_offer_df.iloc[i,column_index] <= offer_end:
                output = 1
                break
        return output


#for each offer_recieved, check if it was viewed using function defined above.

#add column with 1 if offer is viewed, or 0 if not
offer_viewed = []
for i in range(0,len(offers_recieved)):
    offer_viewed.append(check_viewed(offers_recieved['influence_start'].iloc[i],
                                        offers_recieved['influence_end'].iloc[i],
                                        offers_recieved['person'].iloc[i],
                                        offers[offers['event'] == 'offer viewed']))

offers_recieved['offer_viewed'] = pd.Series(offer_viewed)

#for each offer recieved, sum the spending that took place over the course of the offer

spending_during_offer = []
for i in range(0,len(offers_recieved)):
    spending_during_offer.append(spending_in_window(offers_recieved['influence_start'].iloc[i],
                                                    offers_recieved['influence_end'].iloc[i],
                                                    offers_recieved['person'].iloc[i],
                                                    transactions))
    
offers_recieved['spending_during_offer'] = pd.Series(spending_during_offer)

#The above couple of calculations take a while to run, so save result in csv
offers_recieved.to_csv("offers_recieved.csv")

# Cleaning the Data Part III: Spending While Not Under "Offer Influence"

In [5]:
#read in csv from previous step, do some minor cleaning and create data frame to look at just offers that have been viewed
offers_recieved = pd.read_csv('offers_recieved.csv')
offers_recieved = offers_recieved.drop(['id_x', 'id_y','value','channels'], axis=1)
offers_viewed = offers_recieved[offers_recieved['offer_viewed'] == 1]
offers_viewed = offers_viewed.filter(['person','influence_start','influence_end','duration','spending_during_offer']).reset_index()


def during_offer(person,offers_viewed_df,time):
    """flags a transaction if it took place while a person was under the influence of an offer
    returns 1 if yes, it took place during an offer and 0 if not.  
    inputs:
    person: id of person from profile
    offers_viewed: dataframe of offers recived, filtered by those that were viewed
    time: timestamp for offer
    """
    person_df = offers_viewed_df[offers_viewed_df['person'] == person]
    offer_start_index = person_df.columns.get_loc('influence_start')
    offer_end_index = person_df.columns.get_loc('influence_end')
    output = 0
    for i in range(0,len(person_df)):
        if time >= person_df.iloc[i,offer_start_index] and time <= person_df.iloc[i,offer_end_index]:
            output = 1
            break
    return output


#for each transaction label it as having taken place while under the influence of an offer (offer recieved and viewed)
person_index = transactions.columns.get_loc('person')
time_index = transactions.columns.get_loc('time')
during_offer_array = []
for i in range(0,len(transactions)):
    during_offer_array.append(during_offer(transactions.iloc[i,person_index],
                                           offers_viewed,
                                           transactions.iloc[i,time_index]))

transactions['during_offer'] = pd.Series(during_offer_array)    

# Cleaning the Data Part IV: Other Person Level Attributes

In [6]:
#prep df to search over for person Level attributes, only looking at offers that have been viewed to speed up processsing
offers_viewed = offers_recieved[offers_recieved['offer_viewed'] == 1]
offers_viewed = offers_viewed.filter(['person','influence_start','influence_end','duration','spending_during_offer']).reset_index()

def get_profile_transactions(person,transaction_df):
    '''collects information from transactions df about each person in the profile
    returns a list of attributes about persons transaction behavior:
    [count of transactions, count of transaction not under the influence of any offer, sum of the ammount of transactions
    time of first transaction, time of last transaction]
    inputs:
    person: person id from profile
    transaction_df: transaction df to search
    '''
    person_df = transactions[transactions['person'] == person]
    count = len(person_df)
    count_nonoffer = len(person_df) - np.sum(person_df['during_offer'])
    sum_t = np.sum(person_df[person_df['during_offer'] != 1]['amount'])
    if len(person_df) == 0:
        min_time = 0
        max_time = 0
    else:
        min_time = min(person_df['time'])
        max_time = max(person_df['time'])
    return [count,count_nonoffer,sum_t,min_time,max_time]


#for each person in profile df, use get_profile_transactions function to create attributes about people from transactions_df
count, count_nonoffer, sum_amount, min_time, max_time = [],[],[],[],[]
id_index = profile.columns.get_loc('id')
for i in range(0,len(profile)):
    l = get_profile_transactions(profile.iloc[i,id_index],transactions)
    count.append(l[0])
    count_nonoffer.append(l[1])
    sum_amount.append(l[2])
    min_time.append(l[3])
    max_time.append(l[4])

#store attributes in profile df
profile['nonoffer_trans_count'] = pd.Series(count_nonoffer)  
profile['sum_nonoffer_trans'] = pd.Series(sum_amount)  
profile['trans_count'] = pd.Series(count)  
profile['first_trans'] = pd.Series(min_time)  
profile['last_trans'] = pd.Series(max_time)  

In [7]:
def get_profile_offers(person,offers_df):
    '''collects information from offers viewed about each person in the profile
    returns a list of attraibutes about offers a person has recieved as well as their spending:
    [count of offers,sum of the total days the person has been under influence of offer, sum of spending during offers]
    inputs:
    person: person id from profile
    offer_df: offer viewed df to search
    '''
    person_df = offers_df[offers_df['person'] == person]
    num_offers = len(person_df)
    tot_offer_duration = np.sum(person_df['duration'])
    tot_offer_spending = np.sum(person_df['spending_during_offer'])
    return [num_offers,tot_offer_duration,tot_offer_spending]

#collect information about people from offers_viwed df, num offers, sum of offer duration and their total spending during offers
num_offers, tot_offer_duration, tot_offer_spending,  = [],[],[]
id_index = profile.columns.get_loc('id')
for i in range(0,len(profile)):
    l = get_profile_offers(profile.iloc[i,id_index],offers_viewed)
    num_offers.append(l[0])
    tot_offer_duration.append(l[1])
    tot_offer_spending.append(l[2])

#store these attributes in profiles df
profile['num_offers'] = pd.Series(num_offers)  
profile['tot_offer_duration'] = pd.Series(tot_offer_duration)  
profile['tot_offer_spending'] = pd.Series(tot_offer_spending)        

In [8]:
#Need to run some calculations withing profile df to finalize these attributes before merging them with offers recieved

max_length = max(transactions['time'])
#method 1 to define typical spending.  spending w/o offer / (maxtime - duration offers)
profile['typical_spending_m1'] = profile['sum_nonoffer_trans']/(max_length - profile['tot_offer_duration'])
#method 2 to define typical spending.  spending w/o offer / (diff between max/min tranction time - duration offers)
profile['typical_spending_m2'] = profile['sum_nonoffer_trans']/(profile['last_trans'] - profile['first_trans']  - profile['tot_offer_duration'])
#another attribute is how much a person typically spends under offer influence
profile['avg_offer_spending'] = profile['tot_offer_spending']/profile['tot_offer_duration']

#remove some unwanted columns, chose to move forward with typical spending m2
profile_merge = profile.filter(['id','num_offers','typical_spending_m1']).reset_index()

#finally merge profile with offers recieved transactions
offers_recieved = offers_recieved.merge(profile_merge,how='left',left_on='person', right_on='id')

#again save the csv as a check point
offers_recieved.to_csv('offers_recieved.csv')



# Finalize Data to Send to Modeling Stage

In [9]:
#read in csv from previous steps
offers_recieved = pd.read_csv('offers_recieved.csv')

#we only care about the influence of the model when it has been viewed, remove offers that have not be viewed
offers_recieved = offers_recieved[offers_recieved['offer_viewed'] == 1]

columns_to_use = ['offer_type','offer_id','person','time','difficulty','duration','reward','web','email','mobile','social','bogo','discount','informational',
                 'age','income','member_date_int','F','M','income_inpute_flag','typical_spending_m1','num_offers','spending_during_offer']

#filter columns, take one last look at info and send to csv for modeling steps
offers_recieved = offers_recieved.filter(columns_to_use)

offers_recieved.info()

offers_recieved.to_csv('for_model.csv')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17671 entries, 0 to 76271
Data columns (total 22 columns):
offer_type               17671 non-null object
offer_id                 17671 non-null object
person                   17671 non-null object
time                     17671 non-null int64
difficulty               17671 non-null int64
duration                 17671 non-null int64
reward                   17671 non-null int64
web                      17671 non-null int64
email                    17671 non-null int64
mobile                   17671 non-null int64
social                   17671 non-null int64
bogo                     17671 non-null int64
discount                 17671 non-null int64
informational            17671 non-null int64
age                      17671 non-null int64
income                   17671 non-null float64
member_date_int          17671 non-null int64
F                        17671 non-null int64
M                        17671 non-null int64
typical_spen